## Multiple Alignment and Phylogenetic trees¶


### Getting started

1. Enter the "my_notebooks" folder and create a new folder "week06".  
2. In the "my_notebooks/week05" folder, open a notebook from this URL https://raw.githubusercontent.com/hlab1/teaching-fb2023/main/week06/week06_p1_msa_nCov.ipynb.
3. Clear all outputs by "Kernel"->"Restart Kernel and Clear All Outputs".

### Reading a multiple alignment file into R

In the recitation last week, we performed MSA of the coding sequences and amino acid sequences  of coronavirus spike protein.  We provide the MAFFT results of those two MSA in "spike_nt_mafft.fasta" and "spike_aa_mafft.fasta" files. Upload those two files to the "my_notebooks/week06" folder you just created.

spike_nt_mafft.fasta - https://raw.githubusercontent.com/hlab1/teaching-fb2023/main/week06/spike_aa_mafft.fasta

spike_aa_mafft.fasta - https://raw.githubusercontent.com/hlab1/teaching-fb2023/main/week06/spike_nt_mafft.fasta

**Make sure these two files are ending in `.fasta` but not `.fasta.txt`!**

Now we can load the MSA results into R using the `readAAMultipleAlignment` function in the `Biostrings` package.

In [ ]:
library("Biostrings")

In [ ]:
aa_aln <- readAAMultipleAlignment(filepath="spike_aa_mafft.fasta", format="fasta")

What does readAAMultipleAlignment do?

In [ ]:
# Write code below to investigate readAAMultipleAlignment


What is in the variable `aa_aln`?

In [ ]:
# Write code below to investigate aa_aln. First call aa_aln to see what it looks like, then
# use functions defined in the documentation to answer the below questions.


# How many sequences are aligned?


# How many amino acids are in each alignment?


## Building phylogenetic trees for protein sequences¶

We first need to calculate the distance matrix resulting from this alignment. We will use the functions in the "ape" package for this purpose.

First we convert our `aa_aln` object to a "AAbin" format, then use the `dist.aa` function to create a distance matrix.

In [ ]:
library("ape")
aa_bin <- as.AAbin(aa_aln) # Convert the alignment to "AAbin" format

In [ ]:
aa_dist <- dist.aa(aa_bin) # Calculate the genetic distance matrix
as.matrix(aa_dist)

The `nj` function creates a tree from a distance matrix using the Neighbor Joining method.  We can `plot` the tree in multiple ways.

In [ ]:
aa_tree <- nj(aa_dist)

In [ ]:
# What does aa_tree look like?


# What is its type, what is its class?




In [ ]:
plot(aa_tree,'unrooted', use.edge.length=FALSE,cex=0.5,
     main="Unrooted, without branch lengths")

In [ ]:
# 'plot'is the base R plotting function, but this version of 'plot'recognizes the 
# phylo class and calls 'plot.phylo'instead. Look up plot.phylo and see what kind of options
# you can manipulate.



In [ ]:
# Using aa_tree, plot a rooted claodogram that does not use edge length, with a text size of 0.5,
# title of "Cladogram, without branch lengths", and is pointed rightward


Based on the spike protein sequqnce, which SARS coronavirus is the closest to the 2019-nCov isolates?

We can write this tree to a file in the Newick format.

In [ ]:
write.tree(aa_tree,"spike_aa_tree.tre")

## Bootstrapping

The `read.alignment` function in the "seqinr" package creates an alignment object that can be used in the `unrootedNJtree` and `rootedNJtree` function to create a bootstrap tree. 


The `unrootedNJtree` function accepts the alignment and the type of the alignment ("DNA" or "protein").

In [ ]:
library("seqinr")
aa_aln2  <- read.alignment(file = "spike_aa_mafft.fasta", format = "fasta")

In [ ]:
unrootedNJtree <- function(alignment,type) {
     # this function requires the ape and seqinR packages:
     require("ape")
     require("seqinr")
     # define a function for making a tree:
     makemytree <- function(alignmentmat)
     {
        alignment <- ape::as.alignment(alignmentmat)
        if      (type == "protein")
        {
           mydist <- dist.alignment(alignment)
        }
        else if (type == "DNA")
        {
           alignmentbin <- as.DNAbin(alignment)
           mydist <- dist.dna(alignmentbin)
        }
        mytree <- nj(mydist)
        mytree <- makeLabel(mytree, space="") # get rid of spaces in tip names.
        return(mytree)
     }
     # infer a tree
     mymat  <- as.matrix.alignment(alignment)
     mytree <- makemytree(mymat)
     # bootstrap the tree
     myboot <- boot.phylo(mytree, mymat, makemytree)
     # plot the tree:
     plot.phylo(mytree,type="u")   # plot the unrooted phylogenetic tree
     nodelabels(myboot,cex=0.7)    # plot the bootstrap values
     mytree$node.label <- myboot   # make the bootstrap values be the node labels
     return(mytree)
  }

In [ ]:
aa_nj_boot1 <- unrootedNJtree(aa_aln2,type="protein")

The `rootedNJtree` function accepts the alignment, a sequence in the alignment that is used as an "outgroup" to root the tree, and the type of the alignment ("DNA" or "protein").

In [ ]:
rootedNJtree <- function(alignment, theoutgroup, type) {
     # load the ape and seqinR packages:
     require("ape")
     require("seqinr")
     # define a function for making a tree:
     makemytree <- function(alignmentmat, outgroup=`theoutgroup`)
     {
        alignment <- ape::as.alignment(alignmentmat)
        if      (type == "protein")
        {
           mydist <- dist.alignment(alignment)
        }
        else if (type == "DNA")
        {
           alignmentbin <- as.DNAbin(alignment)
           mydist <- dist.dna(alignmentbin)
        }
        mytree <- nj(mydist)
        mytree <- makeLabel(mytree, space="") # get rid of spaces in tip names.
        myrootedtree <- root(mytree, outgroup, r=TRUE)
        return(myrootedtree)
     }
     # infer a tree
     mymat  <- as.matrix.alignment(alignment)
     myrootedtree <- makemytree(mymat, outgroup=theoutgroup)
     # bootstrap the tree
     myboot <- boot.phylo(myrootedtree, mymat, makemytree)
     # plot the tree:
     plot.phylo(myrootedtree, type="p")  # plot the rooted phylogenetic tree
     nodelabels(myboot,cex=0.7)          # plot the bootstrap values
     myrootedtree$node.label <- myboot   # make the bootstrap values be the node labels
     return(myrootedtree)
  }

In [ ]:
aa_nj_boot <- rootedNJtree(aa_aln2,"AY687361.1_SARS_coronavirus_B029_1",type="protein")

### Exercises

1. Try to draw a NJ tree with a different sequence as the outgroup. 
2. Read the DNA sequence alignment in the `spike_nt_mafft.fasta` file and create an unrooted and a rooted tree.  Compare the trees to the trees from the protein sequnce alignments.